In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#url of the arcane fandom page
url="https://arcane.fandom.com/wiki/Category:Characters"

#send a GET request
response=requests.get(url)
if response.status_code==200:
    soup=BeautifulSoup(response.text,"html.parser")

#extract character names and links
    character_data=[]
    characters=soup.find_all("a",class_="category-page__member-link")
    for char in characters:
        name=char.text.strip()
        link=f"https://arcane.fandom.com{char['href']}"

        character_data.append({"Name":name,"Link":link})

#save to a dataframe
    df=pd.DataFrame(character_data)
    df.to_csv("arcane_characters.csv", index=False)
    print("Character data saved to 'arcane_characters.csv'")

else:
    print(f"Failed to retrieve data from the URL. Status code:{response.status_code}")

    

Character data saved to 'arcane_characters.csv'


In [2]:
print(df)


                                      Name  \
0   Category:Alternate Universe characters   
1                                    Amara   
2                          Ambessa Medarda   
3                                  Babette   
4                                    Benzo   
..                                     ...   
82                 Vi and Powder's Parents   
83                   Vi/Alternate Universe   
84                                  Viktor   
85                                 Warwick   
86                            Ximena Talis   

                                                 Link  
0   https://arcane.fandom.com/wiki/Category:Altern...  
1                https://arcane.fandom.com/wiki/Amara  
2      https://arcane.fandom.com/wiki/Ambessa_Medarda  
3              https://arcane.fandom.com/wiki/Babette  
4                https://arcane.fandom.com/wiki/Benzo  
..                                                ...  
82  https://arcane.fandom.com/wiki/Vi_and_Powder%2...  

In [4]:
arcane=pd.read_csv("arcane_characters.csv")
arcane

,Name,Link
0,Category:Alternate Universe characters,https://arcane.fandom.com/wiki/Category:Altern...
1,Amara,https://arcane.fandom.com/wiki/Amara
2,Ambessa Medarda,https://arcane.fandom.com/wiki/Ambessa_Medarda
3,Babette,https://arcane.fandom.com/wiki/Babette
4,Benzo,https://arcane.fandom.com/wiki/Benzo
...,...,...
82,Vi and Powder's Parents,https://arcane.fandom.com/wiki/Vi_and_Powder%2...
83,Vi/Alternate Universe,https://arcane.fandom.com/wiki/Vi/Alternate_Un...
84,Viktor,https://arcane.fandom.com/wiki/Viktor
85,Warwick,https://arcane.fandom.com/wiki/Warwick
